In [38]:
import xarray as xr
import cfgrib
import numpy as np
import xesmf as xe
import eccodes
import salem
%run /mnt/d/work/sunhh/script/function/lambert_ticks.py
%run /mnt/d/work/sunhh/script/function/calculate.py
%run /mnt/d/work/sunhh/script/function/score.py
%run /mnt/d/work/sunhh/script/function/get_wrfvar.py

In [ ]:
import pygrib as pg
import xarray
import numpy as np
import datetime

class grib_decode_helper:
    def __init__(self,grib_paths,lons,lats):
        self.paths = grib_paths
        self.lons = lons
        self.lats = lats

    def get_element_xarray(self,**args):
        all_grb_v = list()
        all_grb_t = list()
        lat1 = float(min(self.lats))
        lat2 = float(max(self.lats))
        lon1 = float(min(self.lons))
        lon2 = float(max(self.lons))
        index = 0
        for p in self.paths:
            index = index+1
            grbs = pg.open(p)
            grb = grbs.select(**args)[0]
            if self.lats is None:
                all_grb_v.append(grb.values)
                lats,lons = grb.latlons()
            else:
                data, lats, lons = grb.data(lat1=lat1,lat2=lat2,lon1=lon1,lon2=lon2)
                all_grb_v.append(data)
            all_grb_t.append(datetime.datetime(grb.year,grb.month,grb.day,grb.hour,grb.minute) )
        return xarray.DataArray(np.array(all_grb_v),coords={'lon':lons[0,:],'lat':lats[:,0],'times':all_grb_t},dims=['times','lat','lon'])



In [3]:
fnl = xr.open_dataset("/mnt/d/work/obs/fnl/0.25/gdas1.fnl0p25.2022072500.f00.grib2",engine="cfgrib", backend_kwargs={'filter_by_keys':{'typeOfLevel': 'isobaricInhPa'}})

skipping variable: paramId==228164 shortName='tcc'
Traceback (most recent call last):
  File "/mnt/d/work/miniconda/envs/sunhh/lib/python3.9/site-packages/cfgrib/dataset.py", line 660, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/mnt/d/work/miniconda/envs/sunhh/lib/python3.9/site-packages/cfgrib/dataset.py", line 591, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  850.,  800.,  750.,  700.,
        650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,  250.,
        200.,  150.,  100.,   70.,   50.,   40.,   30.,   20.,   15.,
         10.,    7.,    5.,    3.,    2.,    1.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  850.,  800.,  750.,  700.,
        650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,  250.,
      

In [5]:
out_grid = xe.util.grid_2d(101.875, 133.125, 0.25, 24.875, 45.125, 0.25).rename_dims({'x':'west_east','y':'south_north'})
ds_wrf = xr.open_dataset("/mnt/d/work/sunhh/data/run2/1.5/fg").rename({'XLONG': 'lon', 'XLAT': 'lat'}).isel(Time=0)
regridder_wrf = xe.Regridder(ds_wrf, out_grid, 'bilinear')

In [39]:
fg_path = "/mnt/d/work/sunhh/data/run2/1.5/fg"
da_path = "/mnt/d/work/sunhh/data/run2/1.5/wrfout_d01_2022-07-25_00_00_00"
levellist = [200,500,700]
u_fg,v_fg,t_fg,rh_fg = get_wrfvar(fg_path,levellist)
u_da,v_da,t_da,rh_da = get_wrfvar(da_path,levellist)

In [41]:
u_fg = regridder_wrf(u_fg)
v_fg = regridder_wrf(v_fg)
t_fg = regridder_wrf(t_fg)
rh_fg = regridder_wrf(rh_fg)
u_da = regridder_wrf(u_da)
v_da = regridder_wrf(v_da)
t_da = regridder_wrf(t_da)
rh_da = regridder_wrf(rh_da)

In [58]:
u_fnl = fnl.u.loc[levellist,25:45:-1,102:133]
v_fnl = fnl.v.loc[levellist,25:45:-1,102:133]
t_fnl = fnl.t.loc[levellist,25:45:-1,102:133]-273.15
rh_fnl = fnl.r.loc[levellist,25:45:-1,102:133]

In [63]:
for level in levellist:
    print(level,"u:",rmse_pe(u_fg.loc[level].values,u_fnl.loc[level].values),rmse_pe(u_da.loc[level].values,u_fnl.loc[level].values))
    print("v:",rmse_pe(v_fg.loc[level].values,v_fnl.loc[level].values),rmse_pe(v_da.loc[level].values,v_fnl.loc[level].values))
    print("t:",rmse_pe(t_fg.loc[level].values,t_fnl.loc[level].values),rmse_pe(t_da.loc[level].values,t_fnl.loc[level].values))
    print("rh:",rmse_pe(rh_fg.loc[level].values,rh_fnl.loc[level].values),rmse_pe(rh_da.loc[level].values,rh_fnl.loc[level].values))

200 u: 1.7515962 1.7911545
v: 2.1769462 2.1371124
t: 0.21995465 0.25623047
rh: 14.947175 14.77847
500 u: 0.8180275 1.7332114
v: 1.2563311 1.7032955
t: 0.20242259 0.28177434
rh: 5.113566 5.105171
700 u: 0.73039454 2.3078542
v: 0.91027856 1.871075
t: 0.19353002 0.6862414
rh: 3.1244876 3.6963983
